In [ ]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import tqdm
# !pip install downcast
from downcast import reduce

In [ ]:
!pip install downcast

In [ ]:
from google.colab import files
files.upload()

Saving kaggle (3).json to kaggle (3).json


{'kaggle (3).json': b'{"username":"shantanuekhande","key":"7c0f27a1e48312dc37124571a15b336e"}'}

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "shantanuekhande"
os.environ['KAGGLE_KEY'] = "7c0f27a1e48312dc37124571a15b336e"
!kaggle competitions download -c 'm5-forecasting-accuracy'  -p 'train'    ## getting all data into train folder 

  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 41.5MB/s]
 57% 9.00M/15.8M [00:00<00:00, 24.1MB/s]
100% 15.8M/15.8M [00:00<00:00, 35.7MB/s]
 58% 9.00M/15.5M [00:00<00:00, 24.7MB/s]
100% 15.5M/15.5M [00:00<00:00, 39.3MB/s]
  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 128MB/s]
 63% 9.00M/14.2M [00:00<00:00, 30.2MB/s]
100% 14.2M/14.2M [00:00<00:00, 40.9MB/s]


In [ ]:
! mkdir final_train

In [ ]:
!7z e /content/train/sales_train_evaluation.csv.zip -o/content/final_train
!7z e /content/train/sales_train_validation.csv.zip -o/content/final_train
!7z e /content/train/sample_submission.csv.zip -o/content/final_train
# !7z e /content/train/calendar.csv.zip -o/content/final_train
!7z e /content/train/sell_prices.csv.zip -o/content/final_train

In [ ]:
sales = pd.read_csv(r'/content/final_train/sales_train_evaluation.csv')
# sales.name = 'sales'
calendar = pd.read_csv(r'/content/train/calendar.csv')
# calendar.name = 'calendar'
prices = pd.read_csv(r'/content/final_train/sell_prices.csv')
# prices.name = 'prices'

In [ ]:
from downcast import reduce

In [ ]:
sales.head(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


In [ ]:
prices.head(5)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [ ]:
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
# replacing NaN value with 'No_event'
calendar.fillna('No_event',inplace=True)
calendar.head(5)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1


In [ ]:
# Reducing memory size of DataFrame using Reduce function from downcasting library.
# https://medium.com/@deepakec1031/how-to-reduce-pandas-data-frame-size-using-downcast-e00cfed8e2c5
sales= reduce(sales)
prices= reduce(prices)
calendar= reduce(calendar)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Time based feature 

### 1.1   Feature name: is_high_sale_months
* in EDA i found that February,march,april, may are the high sale month. so i am going to make one feature to represent high sale month

In [ ]:
f=lambda x: 1 if x==2 or x==3 or x==4 or x==5 else 0
calendar['is_high_sale_months']=calendar['month'].map(f) 
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0


### 1.2 Feature name: is_high_sale_day
* in EDA i found that friday,saturday,sunday are the high sale day in every week.so i am giong to make feature to represent high sale day. 

In [ ]:
f=lambda x: 1 if x==7 or x==1 or x==2 else 0
calendar['is_high_sale_day']=calendar['wday'].map(f) 
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day
0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_event,No_event,No_event,No_event,0,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,No_event,No_event,No_event,No_event,0,0,0,0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,No_event,No_event,No_event,No_event,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,No_event,No_event,No_event,No_event,1,1,0,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,No_event,No_event,No_event,No_event,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,No_event,No_event,No_event,No_event,0,1,1,0,0
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,No_event,No_event,No_event,No_event,0,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,No_event,No_event,No_event,No_event,0,0,0,0,1
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,No_event,No_event,No_event,No_event,0,0,0,0,1


### 1.3 Feature name:is_christmas
* in EDA i found in every christmas sale is almost zero.so i am going to make feature to represent wheatehr day having christmas or not. 

In [ ]:
f=lambda x: 1 if x=='Christmas' else 0
calendar['is_christmas']=calendar['event_name_1'].map(f)
calendar

In [ ]:
calendar[calendar['event_name_1']=='Christmas']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas
330,2011-12-25,11148,Sunday,2,12,2011,d_331,Christmas,National,No_event,No_event,0,0,0,0,1,1
696,2012-12-25,11248,Tuesday,4,12,2012,d_697,Christmas,National,No_event,No_event,0,0,0,0,0,1
1061,2013-12-25,11348,Wednesday,5,12,2013,d_1062,Christmas,National,No_event,No_event,0,0,0,0,0,1
1426,2014-12-25,11447,Thursday,6,12,2014,d_1427,Christmas,National,No_event,No_event,0,0,0,0,0,1
1791,2015-12-25,11547,Friday,7,12,2015,d_1792,Christmas,National,No_event,No_event,0,0,0,0,1,1


## 2. Lags

* in EDA i found that we cant found periodicity in month or in year but we can find the periodicity in weeks.in that concern to capture periodic info "lag" is one technique to capture periodicity. 
* not forget we are going to take lags of target variable. which is "sold"
* lets gather the data into single dataFrame.

In [ ]:
df_sales=pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold')

In [ ]:
df_sales

In [ ]:
df_final= pd.merge(df_sales, calendar, on='d', how='left')

In [ ]:
df_final= pd.merge(df_final, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 

In [ ]:
df_final

* lets understand how feature will get created.and what that feature value represent.
1. lag 1:means how many products have been sold one day before.
2. lag 7:means how many products have been sold seven day before.
3. i am going to creat lag for this many days. lags=[1,7,14,21,28,35,42,49,56,63,70]

### 2.1 Feature Name : ['lag_7', 'lag_14','lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56', 'lag_63','lag_70']

In [ ]:
lags=[1,7,14,21,28,35,42,49,56,63,70]
for i in tqdm.tqdm(lags):
    df_final['lag_'+str(i)]=df_final.groupby(['id'])['sold'].shift(i)

In [ ]:
df_final.head(5)

In [ ]:
df_final.columns

* if you see in dataframe there is lot of NaN value get created. i am replacing NaN value with "0".

In [ ]:
lags=['lag_1','lag_7', 'lag_14','lag_21', 'lag_28', 'lag_35', 'lag_42', 'lag_49', 'lag_56', 'lag_63','lag_70']
for i in lags:
    df_final[i]=df_final[i].fillna(0)

In [ ]:
df_final.head(5)

In [ ]:
with open(r'a_before_rolling.pkl','wb') as f:
    pickle.dump(df_final, f)

In [ ]:
## copy your csv into Google drive
import shutil
# original = r'original path where the file is currently stored\file name.file extension'
# target = r'target path where the file will be copied\file name.file extension'
original=r'/content/a_before_rolling.pkl'
target=r'/content/drive/MyDrive/Project Files/a_before_rolling.pkl'
shutil.copyfile(original, target)

## 3. Rolling Window
* This is another technique to capture perriodicity.
* in this technique we are going to apply some statistical techniques on window.here window mean last some days 
* example if i say window=7 and statistical techniques=mean, then we are going to take last 7 day and apply mean on them  the result will be your feature. we can apply median , min max .. any thing.

* abount Transfom function: https://pbpython.com/images/transform-example.png

* lets understand how this function works.
1. we are grouping the element based on product id .there are 30490 unique ids are there. for each product we have 1941 days of data.
2. so on every product id we are we are appling pandas rolling function.
3. then transfor function will again assign value to DataFrame based on id.
4. for more understanding inhave taken small toy example apply same logic on that.

In [ ]:
df1=pd.read_excel(r"C:\Users\Lenovo\Downloads\sales_transactions.xlsx")

In [ ]:
df1

In [ ]:
df1.groupby('account')['quantity'].transform(lambda s: s.rolling(2).mean())

* appling same logic on real DataFrame

In [ ]:
df_final.head()

In [ ]:
# with open(r'/content/a_before_rolling.pkl', 'rb') as f:
#     df_final = pickle.load(f)

In [ ]:
window=[7,14,28,35,42]
for i in tqdm.tqdm(window):
    df_final['rolling_mean_'+str(i)]=df_final.groupby(['id'])['sold'].transform(lambda s: s.rolling(i).mean())

In [ ]:
# with open(r'b_before_rolling.pkl','wb') as f:
#     pickle.dump(df_final, f)

In [ ]:
df_final.columns

* if you see in dataframe there is lot of NaN value get created. i am replacing NaN value with "0".

In [ ]:
window=['rolling_mean_7','rolling_mean_14', 'rolling_mean_7' ,'rolling_mean_28', 'rolling_mean_35','rolling_mean_42']
for i in window:
    df_final[i]=df_final[i].fillna(0)

In [ ]:
df_final=reduce(df_final)

In [ ]:
df_final.head()

In [ ]:
with open(r'd_before_rolling.pkl','wb') as f:
    pickle.dump(df_final, f)

In [ ]:
# ## copy your csv into Google drive
# import shutil
# # original = r'original path where the file is currently stored\file name.file extension'
# # target = r'target path where the file will be copied\file name.file extension'
# original=r'/content/d_before_rolling.pkl'
# target=r'/content/drive/MyDrive/Project Files/d_before_rolling.pkl'
# shutil.copyfile(original, target)

## 4. Expanding Window

1. again this is one other technique to capture periodicity. 
2. expanding window is same as rolling window only difference is that here window size is not constant its dynamic.

In [ ]:
df_final['expanding_mean']=df_final.groupby(['id'])['sold'].transform(lambda s: s.expanding().mean())

* if you see in dataframe there is lot of NaN value get created. i am replacing NaN value with "0".

In [ ]:
df_final['expanding_mean']=df_final['expanding_mean'].fillna(0)

In [ ]:
df_final=reduce(df_final)

In [ ]:
df_final.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,sell_price,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
with open(r'c_before_rolling.pkl','wb') as f:
    pickle.dump(df_final, f)

## 5. Label Encoding

* in this we are going to convert categorical variable into numeric using sklearn preprocessing library called LabelEncoder 

In [ ]:
from sklearn.preprocessing import LabelEncoder
#https://www.mygreatlearning.com/blog/label-encoding-in-python/
labelencoder=LabelEncoder() 
category=['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id']
for i in tqdm.tqdm(category):
    df_final[i+'_']=labelencoder.fit_transform(df_final[i])

100%|██████████| 10/10 [01:53<00:00, 11.39s/it]


In [ ]:
# with open(r'/content/d_before_rolling.pkl', 'rb') as f:
#     df_final = pickle.load(f)

In [ ]:
df_final=reduce(df_final)

In [ ]:
df_final.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,sell_price,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14370,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No_event,No_event,No_event,No_event,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14380,1438,3,1,0,0


In [ ]:
with open(r'e_before_rolling.pkl','wb') as f:
    pickle.dump(df_final, f)

In [ ]:
# ## mounting google drive
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ## copy your csv into Google drive
# import shutil
# # original = r'original path where the file is currently stored\file name.file extension'
# # target = r'target path where the file will be copied\file name.file extension'
# original=r'/content/b_before_rolling.pkl'
# target=r'/content/drive/MyDrive/Project Files/b_before_rolling.pkl'
# shutil.copyfile(original, target)

'/content/drive/MyDrive/Project Files/b_before_rolling.pkl'

In [ ]:
df_final=df_final.drop(['event_name_1','event_type_1','event_name_2','event_type_2','id','item_id','dept_id','cat_id','store_id','state_id'],axis = 1)

* converting day feature using numeric by removing hypen

In [ ]:
df_final

,d,sold,date,wm_yr_wk,weekday,wday,month,year,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,sell_price,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_
0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14370,1437,3,1,0,0
1,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14380,1438,3,1,0,0
2,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14390,1439,3,1,0,0
3,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14400,1440,3,1,0,0
4,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14410,1441,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,d_1941,1,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,2.980469,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571289,0.643066,0.643066,0.571289,0.571289,0.535645,19,2,3,1,14329,1432,2,0,9,2
59181086,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,2.480469,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.285645,0.142822,0.285645,0.257080,0.261963,0.375000,19,2,3,1,14339,1433,2,0,9,2
59181087,d_1941,2,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,3.980469,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,0.856934,1.000000,0.785645,0.742676,0.785645,0.893555,19,2,3,1,14349,1434,2,0,9,2
59181088,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,1.280273,1.0,1.0,1.0,0.0,3.0,1.0,2.0,1.0,4.0,1.0,3.0,1.857422,1.428711,1.321289,1.286133,1.190430,0.380615,19,2,3,1,14359,1435,2,0,9,2


In [ ]:
l=[]
for i in df_final['d']:
    l.append(i.split('_')[1])
df_final['day']=l
#https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
df_final['day']=df_final['day'].astype(np.int16)

* droping some duplicate feature.

In [ ]:
df_final.head(-1)

,d,sold,date,wm_yr_wk,weekday,wday,month,year,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,sell_price,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14370,1437,3,1,0,0,1
1,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14380,1438,3,1,0,0,1
2,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14390,1439,3,1,0,0,1
3,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14400,1440,3,1,0,0,1
4,d_1,0,2011-01-29,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19,2,3,1,14410,1441,3,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181084,d_1941,4,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,3.980469,3.0,5.0,1.0,5.0,1.0,2.0,4.0,3.0,5.0,3.0,3.0,2.142578,1.928711,2.072266,2.029297,2.119141,3.154297,19,2,3,1,14319,1431,2,0,9,2,1941
59181085,d_1941,1,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,2.980469,1.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571289,0.643066,0.643066,0.571289,0.571289,0.535645,19,2,3,1,14329,1432,2,0,9,2,1941
59181086,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,2.480469,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.285645,0.142822,0.285645,0.257080,0.261963,0.375000,19,2,3,1,14339,1433,2,0,9,2,1941
59181087,d_1941,2,2016-05-22,11617,Sunday,2,5,2016,0,0,0,1,1,0,3.980469,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,0.856934,1.000000,0.785645,0.742676,0.785645,0.893555,19,2,3,1,14349,1434,2,0,9,2,1941


In [ ]:
with open(r'df_final_.pkl','wb') as f:
    pickle.dump(df_final, f)

In [ ]:
## copy your csv into Google drive
import shutil
# original = r'original path where the file is currently stored\file name.file extension'
# target = r'target path where the file will be copied\file name.file extension'
original=r'/content/df_final_.pkl'
target=r'/content/drive/MyDrive/Project Files/df_final_.pkl'
shutil.copyfile(original, target)

'/content/drive/MyDrive/Project Files/df_final_.pkl'

In [ ]:
df_final=df_final.drop(['d','date'],axis=1)

In [ ]:
## copy your csv into Google drive
import shutil
# original = r'original path where the file is currently stored\file name.file extension'
# target = r'target path where the file will be copied\file name.file extension'
original=r'/content/df_final_final.pkl'
target=r'/content/drive/MyDrive/Project Files/df_final_final.pkl'
shutil.copyfile(original, target)

'/content/drive/MyDrive/Project Files/df_final_final.pkl'

In [ ]:
with open(r'df_final_final.pkl','wb') as f:
    pickle.dump(df_final, f)

In [ ]:
df_final=reduce(df_final)

In [ ]:
df_final.head()

,sold,wm_yr_wk,weekday,wday,month,year,snap_CA,snap_TX,snap_WI,is_high_sale_months,is_high_sale_day,is_christmas,sell_price,lag_1,lag_7,lag_14,lag_21,lag_28,lag_35,lag_42,lag_49,lag_56,lag_63,lag_70,rolling_mean_7,rolling_mean_14,rolling_mean_28,rolling_mean_35,rolling_mean_42,expanding_mean,event_name_1_,event_type_1_,event_name_2_,event_type_2_,id_,item_id_,dept_id_,cat_id_,store_id_,state_id_,day
0,0,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14370,1437,3,1,0,0,1
1,0,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14380,1438,3,1,0,0,1
2,0,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14390,1439,3,1,0,0,1
3,0,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14400,1440,3,1,0,0,1
4,0,11101,Saturday,1,1,2011,0,0,0,0,1,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,3,1,14410,1441,3,1,0,0,1
